In [5]:
library(tidyverse)
library(clusterProfiler)
library(BiocParallel)
library(parallel)

# Custom package
library(rutils)

In [6]:
n_cores <- detectCores() - 2
BiocParallel::register(MulticoreParam(n_cores))

In [14]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_path <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

In [15]:
dset_idx <- 1

In [27]:
deg_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_DESeq_results.tsv"))
matrisome_df <- load_matrisome_df(matrisome_path)
gene2id_df <- read_tsv(paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/hugo2entrez.tsv"))

Parsed with column specification:
cols(
  geneID = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double()
)
Parsed with column specification:
cols(
  Division = col_character(),
  Category = col_character(),
  `Gene Symbol` = col_character(),
  `Gene Name` = col_character(),
  Synonyms = col_character(),
  HGNC_IDs = col_double(),
  `HGNC_IDs Links` = col_double(),
  UniProt_IDs = col_character(),
  Refseq_IDs = col_character(),
  Orthology = col_character(),
  Notes = col_character()
)
Parsed with column specification:
cols(
  hugo_symbol = col_character(),
  entrez_gene_id = col_double()
)


In [23]:
colnames(deg_df)

[1] "geneID"         "baseMean"       "log2FoldChange" "lfcSE"         
[5] "stat"           "pvalue"         "padj"

In [32]:
mat_deg_df <- deg_df %>%
    dplyr::filter(padj < 0.05 & abs(log2FoldChange) > log2(2)) %>%
    inner_join(matrisome_df, by = c("geneID" = "gene_symbol")) %>%
    inner_join(gene2id_df, by = c("geneID" = "hugo_symbol")) %>%
    dplyr::select(geneID, entrez_gene_id, everything())

In [34]:
ek_res <- enrichKEGG(gene = mat_deg_df$entrez_gene_id)

In [37]:
emk_res <- enrichMKEGG(gene = mat_deg_df$entrez_gene_id)

No gene set have size > 10 ...
--> return NULL...


In [36]:
head(ek_res)

,ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
hsa04060,hsa04060,Cytokine-cytokine receptor interaction,92/300,295/8081,5.252566e-63,8.824311e-61,5.694888e-61,5617/6367/10563/1437/3586/9518/6358/2920/8995/9560/51561/3627/3458/4283/27189/2919/6352/3557/3624/6355/3553/356/56477/652/656/4803/655/2662/29949/2921/3592/4049/8744/6348/268/6360/6362/4838/6368/6369/3589/2657/6351/3603/2688/4050/55801/8742/3976/6370/6354/84639/3552/1442/8743/7043/7124/11009/6846/9966/3569/151449/6374/6375/2056/5008/5473/6373/7066/6346/3605/3593/651/6364/6349/8600/53342/1896/27190/6372/56832/50604/6361/10673/650/338376/6347/1489/654/6387/6366/2660,92
hsa04061,hsa04061,Viral protein interaction with cytokine and cytokine receptor,41/300,100/8081,4.344925e-33,3.649737e-31,2.355407e-31,6367/10563/3586/6358/2920/9560/3627/4283/2919/6352/6355/56477/29949/2921/4049/6348/6360/6362/6368/6369/6351/6370/6354/8743/7124/11009/6846/3569/6374/6375/5473/6373/6346/6364/6349/6372/50604/6361/6347/6387/6366,41
hsa04512,hsa04512,ECM-receptor interaction,35/300,88/8081,8.825733e-28,4.942410e-26,3.189651e-26,1311/3914/3908/7058/1284/3915/3381/1101/3339/7450/7148/7057/7060/3918/7059/1293/10319/1292/63923/3909/1298/1278/3913/256076/6696/1277/1291/6382/1280/3910/1288/158326/5649/375790/6385,35
hsa04151,hsa04151,PI3K-Akt signaling pathway,59/300,354/8081,1.451899e-23,6.097978e-22,3.935412e-22,5617/2254/2252/80310/1311/3479/1950/3914/3908/7058/374/1284/8822/2255/356/4803/2247/3915/3381/1101/7450/2277/7148/7057/2688/7060/3918/7059/1293/1442/10319/3481/1292/3569/63923/56034/3909/1298/2056/5008/1278/3913/256076/6696/1277/1291/1280/51378/27006/3910/1288/5649/2069/2253/7039/284/9965/3082/4908,59
hsa04974,hsa04974,Protein digestion and absorption,31/300,103/8081,1.773011e-20,5.957318e-19,3.844635e-19,1300/1303/1284/5644/85301/1289/4224/1307/340267/57642/255631/1293/1302/7373/2006/50509/1292/1298/1278/256076/1277/1291/1280/169044/80781/1281/1288/1306/1301/81578/1305,31
hsa05323,hsa05323,Rheumatoid arthritis,28/300,93/8081,1.350303e-18,3.780849e-17,2.440022e-17,1437/2920/51561/3458/2919/6352/3553/2921/6348/3589/4050/1513/3552/7043/7124/4314/3569/6374/4312/3605/6364/6349/8600/6372/10673/6347/6387/284,28
